In [14]:
from skimage.transform import rotate,resize 
from skimage.feature import local_binary_pattern
from skimage import data, io
from skimage.color import label2rgb
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.externals import joblib
import time

In [15]:
def get_files(filename):
    class_train = []
    label_train = []
    for train_class in os.listdir(filename):
        for pic in os.listdir(filename+train_class):
            class_train.append(filename+train_class+'/'+pic)
            label_train.append(train_class)
    temp = np.array([class_train,label_train])
    temp = temp.transpose()
    #shuffle the samples
    np.random.shuffle(temp)
    #after transpose, images is in dimension 0 and label in dimension 1
    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    label_list = [int(i) for i in label_list]
    #print(image_list)
    #print(label_list)
    return image_list,label_list

In [16]:
# settings for LBP
radius = 1
n_points = 8 * radius


#获取label和地址
train_path = 'C:/Users/10449/Desktop/data_svm/train/'
train_image,train_label = get_files(train_path)
test_path = 'C:/Users/10449/Desktop/data_svm/test/'
test_image,test_label = get_files(test_path)

# 读取图像


train_hist = np.zeros( (10000,256) )
test_hist = np.zeros( (1600,256) )
test=np.zeros( (1,256) )

print('开始Lbp特征计算')
for i in range(len(train_image)):
    if i >= len(train_image):
        break
    else:
        image = cv2.imread(train_image[i])
        image=resize(image,(170,170))
        image=image.astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(image, n_points, radius,'default')
        max_bins = int(lbp.max() + 1);
        #hist size:256
        train_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins));


for i in range(len(test_image)):
    if i >= len(test_image):
        break
    else:
        image = cv2.imread(test_image[i])
        image=resize(image,(170,170))
        image=image.astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(image, n_points, radius,'default')
        max_bins = int(lbp.max() + 1);
        #hist size:256
        test_hist[i], _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins));        


print('训练完成')
##训练结果

svr_rbf = SVC(kernel='rbf', C=1e6,gamma=20);
#x=OneVsRestClassifier(svr_rbf,-1).fit(train_hist, train_label).score(test_hist, test_label)
svr_rbf.fit(train_hist, train_label)
joblib.dump(svr_rbf, "C:/Users/10449/Desktop/train.model")
#print(x)

test_path="C:/Users/10449/Desktop/200.jpg"
print('开始测试')
t1=time.time()
image = cv2.imread(test_path)
image=resize(image,(256,256))
image=image.astype(np.float32)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
lbp = local_binary_pattern(image, n_points, radius,'default')
#max_bins = int(lbp.max() + 1);
        #hist size:256
#test, _ = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins)); 
r1=svr_rbf.predict(lbp)
t2=time.time()
print(t2-t1)
RF=joblib.load('C:/Users/10449/Desktop/train.model')
result=RF.predict(lbp)
t2=time.time()
print(t2-t1)

开始Lbp特征计算


C:\Users\10449\Anaconda3\envs\py35\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\10449\Anaconda3\envs\py35\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


训练完成
开始测试
0.3917994499206543
0.7655842304229736
